In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from transformers import BertTokenizer

# Load your dataset

In [40]:
# Load your dataset
data = pd.read_excel('RAMI salon.xlsx')

# Assuming the dataset contains 'Text', 'Emoji', and 'Label' columns

In [ ]:
# Modify according to the actual column names in 'RAMI salon.xlsx'
texts = data['Text'].tolist()  # Column for BERT text input
emojis = data['Emoji'].tolist()  # Column for emoji indices (for LSTM)
labels = data['Label'].tolist()  # Column for target labels

# Prepare text and emoji inputs

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

C:\Users\Ninduki\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# Rename based on the actual column name you found in the previous step
data.rename(columns={'Review ': 'Review'}, inplace=True)

NameError: name 'data' is not defined

# Tokenize the text for BERT

In [5]:
# Replace 'Review' with the correct column name (e.g., 'Sentiment')
data['input_ids'] = data['Sentiment'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=64, truncation=True, padding='max_length'))
data['attention_mask'] = data['input_ids'].apply(lambda x: [1 if i > 0 else 0 for i in x])

NameError: name 'data' is not defined

In [6]:
# Inspect the first few rows of the dataset
print(data.columns)
print(data.head())


NameError: name 'data' is not defined

In [51]:
# Load the dataset
data_path = 'Emoji_Sentiment_Data_200.csv'
data = pd.read_csv(data_path)

# Print the column names
print(data.columns)


Index(['Emoji', 'Sentiment'], dtype='object')


In [52]:
from sklearn.preprocessing import LabelEncoder

# Encode the 'Emoji' column
emoji_encoder = LabelEncoder()
data['Emoji_index'] = emoji_encoder.fit_transform(data['Emoji'])

# Verify that 'Emoji_index' has been added
print(data.head())


  Emoji Sentiment  Emoji_index
0     😀  positive           92
1     😃  positive           95
2     😄  positive           96
3     😁  positive           93
4     😆  positive           98


# Encode emojis to indices (if emojis are categorical)

In [53]:
# Clean column names to remove any potential trailing spaces
data.columns = data.columns.str.strip()

# Now fit the LabelEncoder
emoji_encoder = LabelEncoder()
data['Emoji_index'] = emoji_encoder.fit_transform(data['Emoji'])


# Encode sentiments to labels (assuming classification task)

In [54]:
# Encode the 'Sentiment' column
label_encoder = LabelEncoder()
data['Sentiment_label'] = label_encoder.fit_transform(data['Sentiment'])


# Prepare datasets

In [58]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the 'Sentiment' column (or correct column if different)
data['input_ids'] = data['Sentiment'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=64, truncation=True, padding='max_length'))
data['attention_mask'] = data['input_ids'].apply(lambda x: [1 if i > 0 else 0 for i in x])

# Encode the 'Sentiment' column into numerical values
label_encoder = LabelEncoder()
data['Sentiment_encoded'] = label_encoder.fit_transform(data['Sentiment'])

# Extract data for model training
X = data['Emoji_index'].values
y = data['Sentiment_encoded'].values

# Convert numerical sentiment labels to one-hot encoding
y = to_categorical(y)

# Now y will contain one-hot encoded labels


C:\Users\Micheal\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Train-test split

In [63]:
# Drop any rows with missing data in the relevant columns
# Ensure input_ids, attention_mask, Emoji_index, and Sentiment_encoded are aligned
data_cleaned = data.dropna(subset=['input_ids', 'attention_mask', 'Emoji_index', 'Sentiment_encoded'])

# Recreate the arrays after cleaning to ensure consistent lengths
X_input_ids = np.array(data_cleaned['input_ids'].tolist())
X_attention_masks = np.array(data_cleaned['attention_mask'].tolist())
X_emoji_indices = np.array(data_cleaned['Emoji_index'].values)
y = np.array(data_cleaned['Sentiment_encoded'].values)

# Now ensure consistent lengths
print(f"X_input_ids shape: {X_input_ids.shape}")
print(f"X_attention_masks shape: {X_attention_masks.shape}")
print(f"X_emoji_indices shape: {X_emoji_indices.shape}")
print(f"y shape: {y.shape}")

# Ensure all arrays have the same number of rows
if len(X_input_ids) == len(X_attention_masks) == len(X_emoji_indices) == len(y):
    # Proceed with train-test split
    X_train_ids, X_val_ids, X_train_masks, X_val_masks, X_train_emojis, X_val_emojis, y_train, y_val = train_test_split(
        X_input_ids, X_attention_masks, X_emoji_indices, y, test_size=0.2, random_state=42
    )

    # Convert to TensorDatasets
    train_data = TensorDataset(torch.tensor(X_train_ids), torch.tensor(X_train_masks), torch.tensor(X_train_emojis), torch.tensor(y_train))
    val_data = TensorDataset(torch.tensor(X_val_ids), torch.tensor(X_val_masks), torch.tensor(X_val_emojis), torch.tensor(y_val))

    # DataLoader
    train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=32)
    validation_dataloader = DataLoader(val_data, sampler=RandomSampler(val_data), batch_size=32)
else:
    print("Error: Arrays still have inconsistent lengths after cleaning.")


X_input_ids shape: (236, 64)
X_attention_masks shape: (236, 64)
X_emoji_indices shape: (236,)
y shape: (236,)


# Define the combined model

In [2]:
class BERTLSTMModel(nn.Module):
    def __init__(self, bert_model_name, max_emojis, embedding_size, lstm_hidden_size, num_labels):
        super(BERTLSTMModel, self).__init__()
        
        # BERT model for sequence classification
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.3)

        # Emoji Embedding and LSTM model
        self.emoji_embedding = nn.Embedding(num_embeddings=max_emojis, embedding_dim=embedding_size)
        self.lstm = nn.LSTM(embedding_size, lstm_hidden_size, batch_first=True)
        
        # Classifier
        self.classifier = nn.Linear(self.bert.config.hidden_size + lstm_hidden_size, num_labels)
    
    def forward(self, input_ids, attention_mask, emoji_indices):
        # Get BERT embeddings
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_pooled_output = bert_outputs.pooler_output
        bert_pooled_output = self.dropout(bert_pooled_output)
        
        # Ensure emoji_indices is 2D (batch_size, sequence_length)
        if emoji_indices.dim() == 1:
            emoji_indices = emoji_indices.unsqueeze(1)  # Add a sequence length dimension if necessary
        
        # Get emoji embeddings and pass through LSTM
        emoji_embeds = self.emoji_embedding(emoji_indices)
        print(f"emoji_embeds shape: {emoji_embeds.shape}")  # Debugging shape of emoji embeddings
        
        lstm_out, _ = self.lstm(emoji_embeds)
        print(f"lstm_out shape: {lstm_out.shape}")  # Debugging shape of LSTM output
        
        # Handle both 2D and 3D outputs
        if lstm_out.dim() == 3:
            lstm_out = lstm_out[:, -1, :]  # Take the last output of the LSTM (3D case)
        elif lstm_out.dim() == 2:
            # In case of 2D output, handle directly
            pass  # lstm_out is already 2D, no need for further indexing
        
        # Concatenate BERT and LSTM outputs
        combined_output = torch.cat((bert_pooled_output, lstm_out), dim=1)
        
        # Pass through the classifier
        logits = self.classifier(combined_output)
        
        return logits


NameError: name 'nn' is not defined

# Optimizer and scheduler

In [3]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


NameError: name 'AdamW' is not defined

# Training loop

In [4]:
for epoch_i in range(0, epochs):
    print(f'======== Epoch {epoch_i + 1} / {epochs} ========')
    print('Training...')

    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_emoji_indices = batch[2].to(device)
        b_labels = batch[3].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, attention_mask=b_attention_mask, emoji_indices=b_emoji_indices)
        
        loss = nn.CrossEntropyLoss()(outputs, b_labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"  Average training loss: {avg_train_loss:.2f}")

    # Validation
    print("Running Validation...")
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_emoji_indices = batch[2].to(device)
        b_labels = batch[3].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_attention_mask, emoji_indices=b_emoji_indices)

        loss = nn.CrossEntropyLoss()(outputs, b_labels)
        total_eval_loss += loss.item()

        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        labels = b_labels.cpu().numpy()

        total_eval_accuracy += np.sum(preds == labels) / len(labels)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    print(f"  Accuracy: {avg_val_accuracy:.2f}")
    print(f"  Validation Loss: {avg_val_loss:.2f}")

print("Training complete!")

NameError: name 'epochs' is not defined

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

# Load your dataset (update the file path as necessary)
data_path = 'Emoji_Sentiment_Data_200.csv'
data = pd.read_csv(data_path)

# Clean column names to remove trailing spaces
data.columns = data.columns.str.strip()

# Encode 'Emoji' and 'Sentiment' columns
emoji_encoder = LabelEncoder()
data['Emoji_index'] = emoji_encoder.fit_transform(data['Emoji'])

label_encoder = LabelEncoder()
data['Sentiment_encoded'] = label_encoder.fit_transform(data['Sentiment'])

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the 'Sentiment' column for BERT
data['input_ids'] = data['Sentiment'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=64, truncation=True, padding='max_length'))
data['attention_mask'] = data['input_ids'].apply(lambda x: [1 if i > 0 else 0 for i in x])

# Drop rows with missing data
data_cleaned = data.dropna(subset=['input_ids', 'attention_mask', 'Emoji_index', 'Sentiment_encoded'])

# Convert columns to numpy arrays
X_input_ids = np.array(data_cleaned['input_ids'].tolist())
X_attention_masks = np.array(data_cleaned['attention_mask'].tolist())
X_emoji_indices = np.array(data_cleaned['Emoji_index'].values)
y = to_categorical(data_cleaned['Sentiment_encoded'].values)

# Train-test split
X_train_ids, X_val_ids, X_train_masks, X_val_masks, X_train_emojis, X_val_emojis, y_train, y_val = train_test_split(
    X_input_ids, X_attention_masks, X_emoji_indices, y, test_size=0.2, random_state=42
)

# Convert to TensorDatasets
train_data = TensorDataset(torch.tensor(X_train_ids), torch.tensor(X_train_masks), torch.tensor(X_train_emojis), torch.tensor(y_train))
val_data = TensorDataset(torch.tensor(X_val_ids), torch.tensor(X_val_masks), torch.tensor(X_val_emojis), torch.tensor(y_val))

# DataLoader
train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=32)
validation_dataloader = DataLoader(val_data, sampler=RandomSampler(val_data), batch_size=32)

# Define the BERT-LSTM model
class BERTLSTMModel(nn.Module):
    def __init__(self, bert_model_name, max_emojis, embedding_size, lstm_hidden_size, num_labels):
        super(BERTLSTMModel, self).__init__()
        
        # BERT model for sequence classification
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.3)

        # Emoji Embedding and LSTM model
        self.emoji_embedding = nn.Embedding(num_embeddings=max_emojis, embedding_dim=embedding_size)
        self.lstm = nn.LSTM(embedding_size, lstm_hidden_size, batch_first=True)
        
        # Classifier
        self.classifier = nn.Linear(self.bert.config.hidden_size + lstm_hidden_size, num_labels)
    
    def forward(self, input_ids, attention_mask, emoji_indices):
        # BERT embeddings
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_pooled_output = bert_outputs.pooler_output
        bert_pooled_output = self.dropout(bert_pooled_output)
        
        # Emoji LSTM embeddings
        emoji_embeds = self.emoji_embedding(emoji_indices)
        lstm_out, _ = self.lstm(emoji_embeds)
        lstm_out = lstm_out[:, -1, :]  # Take the last LSTM output

        # Concatenate BERT and LSTM outputs
        combined_output = torch.cat((bert_pooled_output, lstm_out), dim=1)
        
        # Classification
        logits = self.classifier(combined_output)
        
        return logits

# Instantiate the model
model = BERTLSTMModel('bert-base-uncased', max_emojis=len(emoji_encoder.classes_), embedding_size=50, lstm_hidden_size=50, num_labels=y.shape[1])

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch_i in range(0, epochs):
    print(f'======== Epoch {epoch_i + 1} / {epochs} ========')
    print('Training...')

    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_emoji_indices = batch[2].to(device)
        b_labels = batch[3].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, attention_mask=b_attention_mask, emoji_indices=b_emoji_indices)
        loss = nn.CrossEntropyLoss()(outputs, b_labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"  Average training loss: {avg_train_loss:.2f}")

    # Validation loop
    print("Running Validation...")
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_emoji_indices = batch[2].to(device)
        b_labels = batch[3].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_attention_mask, emoji_indices=b_emoji_indices)

        loss = nn.CrossEntropyLoss()(outputs, b_labels)
        total_eval_loss += loss.item()

        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        labels = b_labels.cpu().numpy()
        total_eval_accuracy += np.sum(preds == labels) / len(labels)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    print(f"  Accuracy: {avg_val_accuracy:.2f}")
    print(f"  Validation Loss: {avg_val_loss:.2f}")

print("Training complete!")


C:\Users\sachi\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sachi\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\sachi\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimizatio

======== Epoch 1 / 10 ========
Training...


IndexError: too many indices for tensor of dimension 2

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

C:\Users\sachi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load your dataset (update the file path as necessary)
data_path = 'Emoji_Sentiment_Data_200.csv'
data = pd.read_csv(data_path)

# Clean column names to remove trailing spaces
data.columns = data.columns.str.strip()

# Encode 'Emoji' and 'Sentiment' columns
emoji_encoder = LabelEncoder()
data['Emoji_index'] = emoji_encoder.fit_transform(data['Emoji'])

label_encoder = LabelEncoder()
data['Sentiment_encoded'] = label_encoder.fit_transform(data['Sentiment'])

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [3]:
# Tokenize the 'Sentiment' column for BERT
data['input_ids'] = data['Sentiment'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=64, truncation=True, padding='max_length'))
data['attention_mask'] = data['input_ids'].apply(lambda x: [1 if i > 0 else 0 for i in x])

# Drop rows with missing data
data_cleaned = data.dropna(subset=['input_ids', 'attention_mask', 'Emoji_index', 'Sentiment_encoded'])

# Convert columns to numpy arrays
X_input_ids = np.array(data_cleaned['input_ids'].tolist())
X_attention_masks = np.array(data_cleaned['attention_mask'].tolist())
X_emoji_indices = np.array(data_cleaned['Emoji_index'].values)
y = to_categorical(data_cleaned['Sentiment_encoded'].values)

# Train-test split
X_train_ids, X_val_ids, X_train_masks, X_val_masks, X_train_emojis, X_val_emojis, y_train, y_val = train_test_split(
    X_input_ids, X_attention_masks, X_emoji_indices, y, test_size=0.2, random_state=42
)

# Convert to TensorDatasets
train_data = TensorDataset(torch.tensor(X_train_ids), torch.tensor(X_train_masks), torch.tensor(X_train_emojis), torch.tensor(y_train))
val_data = TensorDataset(torch.tensor(X_val_ids), torch.tensor(X_val_masks), torch.tensor(X_val_emojis), torch.tensor(y_val))

# DataLoader
train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=32)
validation_dataloader = DataLoader(val_data, sampler=RandomSampler(val_data), batch_size=32)

In [5]:
# Define the BERT-LSTM model
class BERTLSTMModel(nn.Module):
    def __init__(self, bert_model_name, max_emojis, embedding_size, lstm_hidden_size, num_labels):
        super(BERTLSTMModel, self).__init__()
        
        # BERT model for sequence classification
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.3)

        # Emoji Embedding and LSTM model
        self.emoji_embedding = nn.Embedding(num_embeddings=max_emojis, embedding_dim=embedding_size)
        self.lstm = nn.LSTM(embedding_size, lstm_hidden_size, batch_first=True)
        
        # Classifier
        self.classifier = nn.Linear(self.bert.config.hidden_size + lstm_hidden_size, num_labels)
    
    def forward(self, input_ids, attention_mask, emoji_indices):
        # BERT embeddings
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_pooled_output = bert_outputs.pooler_output
        bert_pooled_output = self.dropout(bert_pooled_output)
        
        # Ensure emoji_indices is 2D (batch_size, sequence_length)
        if emoji_indices.dim() == 1:
            emoji_indices = emoji_indices.unsqueeze(1)  # Add a sequence length dimension if necessary

        # Emoji LSTM embeddings
        emoji_embeds = self.emoji_embedding(emoji_indices)
        
        # Check the shape of the emoji embeddings
        print(f"emoji_embeds shape: {emoji_embeds.shape}")  # Debugging output

        # Pass through LSTM
        lstm_out, _ = self.lstm(emoji_embeds)
        
        # If LSTM output is 3D, take the last LSTM output; otherwise, handle 2D output
        if lstm_out.dim() == 3:
            lstm_out = lstm_out[:, -1, :]  # Take the last output in the sequence (3D case)
        elif lstm_out.dim() == 2:
            pass  # LSTM output is already 2D, no further indexing needed

        # Concatenate BERT and LSTM outputs
        combined_output = torch.cat((bert_pooled_output, lstm_out), dim=1)
        
        # Classification
        logits = self.classifier(combined_output)
        
        return logits

In [7]:






# Instantiate the model
model = BERTLSTMModel('bert-base-uncased', max_emojis=len(emoji_encoder.classes_), embedding_size=50, lstm_hidden_size=50, num_labels=y.shape[1])

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch_i in range(0, epochs):
    print(f'======== Epoch {epoch_i + 1} / {epochs} ========')
    print('Training...')

    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_emoji_indices = batch[2].to(device)
        b_labels = batch[3].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, attention_mask=b_attention_mask, emoji_indices=b_emoji_indices)
        loss = nn.CrossEntropyLoss()(outputs, b_labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"  Average training loss: {avg_train_loss:.2f}")

    # Validation loop
print("Running Validation...")
model.eval()

total_eval_accuracy = 0
total_eval_loss = 0

for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_attention_mask = batch[1].to(device)
    b_emoji_indices = batch[2].to(device)
    b_labels = batch[3].to(device)

    with torch.no_grad():
        outputs = model(b_input_ids, attention_mask=b_attention_mask, emoji_indices=b_emoji_indices)

    loss = nn.CrossEntropyLoss()(outputs, b_labels)
    total_eval_loss += loss.item()

    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = b_labels.cpu().numpy()

    # Convert one-hot encoded labels back to their integer form
    labels = np.argmax(labels, axis=1)

    # Now compare preds and labels
    total_eval_accuracy += np.sum(preds == labels) / len(labels)

avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
avg_val_loss = total_eval_loss / len(validation_dataloader)
print(f"  Accuracy: {avg_val_accuracy:.2f}")
print(f"  Validation Loss: {avg_val_loss:.2f}")


print("Training complete!")


======== Epoch 1 / 10 ========
Training...
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([28, 1, 50])
  Average training loss: 1.05
======== Epoch 2 / 10 ========
Training...
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([28, 1, 50])
  Average training loss: 0.80
======== Epoch 3 / 10 ========
Training...
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([32, 1, 50])
emoji_embeds shape: torch.Size([28, 1, 50])
  Average training 

In [8]:
# Save the model
model_save_path = "bert_lstm_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to bert_lstm_model.pth


In [9]:
import torch
import numpy as np
from transformers import BertTokenizer

# Dummy text and emoji data for testing
dummy_texts = [
    "I love this!",  # positive
    "This is the worst",  # negative
    "I feel neutral about this",  # neutral
]

dummy_emojis = ["😀", "😡", "😐"]  # Positive, Negative, Neutral emojis

# Dummy labels (optional, only for comparison in this case)
dummy_labels = [
    [1, 0, 0],  # positive
    [0, 1, 0],  # negative
    [0, 0, 1],  # neutral
]

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dummy texts
input_ids = [tokenizer.encode(text, add_special_tokens=True, max_length=64, truncation=True, padding='max_length') for text in dummy_texts]
attention_masks = [[1 if token > 0 else 0 for token in tokens] for tokens in input_ids]

# Emoji dictionary (same as used during training)
emoji_dict = {"😀": 0, "😡": 1, "😐": 2}  # Example emoji to index mapping
emoji_indices = [emoji_dict[emoji] for emoji in dummy_emojis]

# Convert to tensors
X_input_ids = torch.tensor(input_ids)
X_attention_masks = torch.tensor(attention_masks)
X_emoji_indices = torch.tensor(emoji_indices)
y_labels = torch.tensor(dummy_labels)  # Optional: for comparison if you want

# Load your model architecture (same as during training)
model = BERTLSTMModel('bert-base-uncased', max_emojis=len(emoji_dict), embedding_size=50, lstm_hidden_size=50, num_labels=len(dummy_labels[0]))

# Load the model's saved state
model_save_path = "bert_lstm_model.pth"
model.load_state_dict(torch.load(model_save_path))

# Move the model to CPU/GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Move the input tensors to the same device as the model
X_input_ids = X_input_ids.to(device)
X_attention_masks = X_attention_masks.to(device)
X_emoji_indices = X_emoji_indices.to(device)

# Set the model to evaluation mode
model.eval()

# Run the model on the dummy data
with torch.no_grad():
    outputs = model(X_input_ids, attention_mask=X_attention_masks, emoji_indices=X_emoji_indices)

# Get the predictions
preds = torch.argmax(outputs, dim=1).cpu().numpy()

# (Optional) If comparing with true labels:
true_labels = np.argmax(y_labels.cpu().numpy(), axis=1)

# Print the results
for i, text in enumerate(dummy_texts):
    print(f"Text: {text}")
    print(f"Emoji: {dummy_emojis[i]}")
    print(f"Predicted label: {preds[i]} (0=Positive, 1=Negative, 2=Neutral)")
    if dummy_labels:
        print(f"True label: {true_labels[i]}")
    print()


C:\Users\sachi\AppData\Local\Temp\ipykernel_13872\2469752451.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))


RuntimeError: Error(s) in loading state_dict for BERTLSTMModel:
	size mismatch for emoji_embedding.weight: copying a param with shape torch.Size([223, 50]) from checkpoint, the shape in current model is torch.Size([3, 50]).

In [10]:
import torch
import numpy as np
from transformers import BertTokenizer

# Dummy text and emoji data for testing
dummy_texts = [
    "I love this!",  # positive
    "This is the worst",  # negative
    "I feel neutral about this",  # neutral
]

dummy_emojis = ["😀", "😡", "😐"]  # Positive, Negative, Neutral emojis

# Dummy labels (optional, only for comparison in this case)
dummy_labels = [
    [1, 0, 0],  # positive
    [0, 1, 0],  # negative
    [0, 0, 1],  # neutral
]

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dummy texts
input_ids = [tokenizer.encode(text, add_special_tokens=True, max_length=64, truncation=True, padding='max_length') for text in dummy_texts]
attention_masks = [[1 if token > 0 else 0 for token in tokens] for tokens in input_ids]

# Emoji dictionary (same as used during training)
emoji_dict = {"😀": 0, "😡": 1, "😐": 2}  # Example emoji to index mapping
emoji_indices = [emoji_dict[emoji] for emoji in dummy_emojis]

# Convert to tensors
X_input_ids = torch.tensor(input_ids)
X_attention_masks = torch.tensor(attention_masks)
X_emoji_indices = torch.tensor(emoji_indices)
y_labels = torch.tensor(dummy_labels)  # Optional: for comparison if you want

# Load your model architecture with the current number of emojis (3 emojis)
model = BERTLSTMModel('bert-base-uncased', max_emojis=3, embedding_size=50, lstm_hidden_size=50, num_labels=len(dummy_labels[0]))

# Load the model's saved state with strict=False (ignores size mismatch)
model_save_path = "bert_lstm_model.pth"
model.load_state_dict(torch.load(model_save_path), strict=False)

# Move the model to CPU/GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Move the input tensors to the same device as the model
X_input_ids = X_input_ids.to(device)
X_attention_masks = X_attention_masks.to(device)
X_emoji_indices = X_emoji_indices.to(device)

# Set the model to evaluation mode
model.eval()

# Run the model on the dummy data
with torch.no_grad():
    outputs = model(X_input_ids, attention_mask=X_attention_masks, emoji_indices=X_emoji_indices)

# Get the predictions
preds = torch.argmax(outputs, dim=1).cpu().numpy()

# (Optional) If comparing with true labels:
true_labels = np.argmax(y_labels.cpu().numpy(), axis=1)

# Print the results
for i, text in enumerate(dummy_texts):
    print(f"Text: {text}")
    print(f"Emoji: {dummy_emojis[i]}")
    print(f"Predicted label: {preds[i]} (0=Positive, 1=Negative, 2=Neutral)")
    if dummy_labels:
        print(f"True label: {true_labels[i]}")
    print()


C:\Users\sachi\AppData\Local\Temp\ipykernel_13872\1537872983.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path), strict=F

RuntimeError: Error(s) in loading state_dict for BERTLSTMModel:
	size mismatch for emoji_embedding.weight: copying a param with shape torch.Size([223, 50]) from checkpoint, the shape in current model is torch.Size([3, 50]).

In [21]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup, BertModel
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, random_split
from transformers import BertForSequenceClassification

In [22]:
# Load your datasets (update with your actual file paths)
salon_review_path = 'RAMI salon.xlsx'
emoji_data_path = 'Emoji_Sentiment_Data_200.csv'

# Load salon review data and emoji sentiment data
salon_reviews = pd.read_excel(salon_review_path)
emoji_data = pd.read_csv(emoji_data_path)

# Check the column names of the salon_reviews dataframe
print("Columns in salon reviews dataset: ", salon_reviews.columns)

# Preprocessing for the salon reviews based on available columns
# Dynamically select the columns if they exist
expected_columns = ['salon_name', 'beautician', 'Service_type', 'Review']
available_columns = [col for col in expected_columns if col in salon_reviews.columns]
salon_reviews = salon_reviews[available_columns]

# Preprocessing for emoji dataset
emoji_data = emoji_data[['Emoji', 'Sentiment']]

# Encode emoji sentiment
emoji_encoder = LabelEncoder()
emoji_data['Emoji_index'] = emoji_encoder.fit_transform(emoji_data['Emoji'])

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize reviews
def preprocess_reviews(reviews):
    input_ids = []
    attention_masks = []

    for review in reviews:
        encoded = tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=64,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Extract and preprocess reviews
input_ids, attention_masks = preprocess_reviews(salon_reviews['Review'].values)

# Map emojis in reviews to their sentiment index
def extract_emojis_from_review(review, emoji_data):
    emojis = [char for char in review if char in emoji_data['Emoji'].values]
    if emojis:
        return emoji_data[emoji_data['Emoji'] == emojis[0]]['Emoji_index'].values[0]
    return emoji_encoder.transform(['😊'])[0]  # Default emoji if none found

salon_reviews['Emoji_index'] = salon_reviews['Review'].apply(lambda x: extract_emojis_from_review(x, emoji_data))

Columns in salon reviews dataset:  Index(['salon_name', 'beautician ', 'Service_type', 'Review', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'],
      dtype='object')


In [23]:
# Convert emojis and labels to tensors
emoji_indices = torch.tensor(salon_reviews['Emoji_index'].values)
labels = torch.tensor([1 if 'positive' in review.lower() else 0 for review in salon_reviews['Review'].values])

# Create TensorDataset and DataLoader
dataset = TensorDataset(input_ids, attention_masks, emoji_indices, labels)

# Split dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=8)
validation_dataloader = DataLoader(val_dataset, sampler=RandomSampler(val_dataset), batch_size=8)



In [39]:
from transformers import BertForSequenceClassification

class BERTLSTMWithSentiment(nn.Module):
    def __init__(self, bert_model_name, max_emojis, embedding_size, lstm_hidden_size):
        super(BERTLSTMWithSentiment, self).__init__()
        
        # BERT for text classification (sentiment)
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.sentiment_model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

        self.dropout = nn.Dropout(0.3)
        self.bert_to_lstm = nn.Linear(self.bert.config.hidden_size, lstm_hidden_size)
        
        self.emoji_embedding = nn.Embedding(num_embeddings=max_emojis, embedding_dim=embedding_size)
        self.lstm = nn.LSTM(embedding_size, lstm_hidden_size, batch_first=True)
        
        self.text_weight = nn.Parameter(torch.tensor(0.85))  
        self.emoji_weight = nn.Parameter(torch.tensor(0.15))  
        
        self.regressor = nn.Linear(lstm_hidden_size + 1, 1)  # +1 for sentiment score
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask, emoji_indices):
        # BERT for text embeddings
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_pooled_output = bert_outputs.pooler_output
        bert_pooled_output = self.dropout(bert_pooled_output)
        bert_pooled_output = self.bert_to_lstm(bert_pooled_output)
        
        # BERT for sentiment
        sentiment_outputs = self.sentiment_model(input_ids=input_ids, attention_mask=attention_mask)
        sentiment_score = torch.softmax(sentiment_outputs.logits, dim=1)[:, -1].unsqueeze(1)  # Take the negative sentiment score
        
        # Emoji embeddings and LSTM
        emoji_embeds = self.emoji_embedding(emoji_indices)
        lstm_out, _ = self.lstm(emoji_embeds)
        lstm_out = lstm_out[:, -1, :]
        
        # Combine text, emoji, and sentiment score
        combined_output = self.text_weight * bert_pooled_output + self.emoji_weight * lstm_out
        combined_output = torch.cat((combined_output, sentiment_score), dim=1)  # Include sentiment score
        
        # Final prediction
        score = self.regressor(combined_output)
        score = self.sigmoid(score) * 5  # Scale to 0-5
        
        return score




# Model parameters
embedding_size = 50
lstm_hidden_size = 50
num_labels = 2
max_emojis = len(emoji_encoder.classes_)

# Load the pre-trained sentiment classification model, ignoring size mismatch for the classifier
model = BertForSequenceClassification.from_pretrained(
    'nlptown/bert-base-multilingual-uncased-sentiment', 
    num_labels=1,  # Modify for your regression task
    ignore_mismatched_sizes=True  # Ignore size mismatch for the classifier
)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [40]:
# Model parameters
embedding_size = 50
lstm_hidden_size = 50
max_emojis = len(emoji_encoder.classes_)  # Assuming you have emoji encoder

# Initialize the model with BERT and LSTM for emoji
model = BERTLSTMModel('bert-base-uncased', max_emojis=max_emojis, embedding_size=embedding_size, lstm_hidden_size=lstm_hidden_size)

# Move model to device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Loss function for regression (MSELoss for continuous scores)
loss_fn = nn.MSELoss()

# Training Loop
for epoch_i in range(epochs):
    print(f'======== Epoch {epoch_i + 1} / {epochs} ========')
    print('Training...')
    
    total_train_loss = 0
    model.train()  # Set model to train mode
    
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_emoji_indices = batch[2].to(device)
        b_labels = batch[3].to(device).float()  # Ensure labels are float for regression

        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_attention_mask, emoji_indices=b_emoji_indices)
        
        # Compute the loss (MSE for regression)
        loss = loss_fn(outputs.squeeze(), b_labels)  # Squeeze to handle dimensions
        total_train_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"  Average training loss: {avg_train_loss:.2f}")

    # Validation loop
    print("Running Validation...")
    model.eval()  # Set model to evaluation mode

    total_eval_loss = 0
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_emoji_indices = batch[2].to(device)
        b_labels = batch[3].to(device).float()

        with torch.no_grad():
            # Forward pass
            outputs = model(b_input_ids, attention_mask=b_attention_mask, emoji_indices=b_emoji_indices)

        # Compute validation loss (MSE)
        loss = loss_fn(outputs.squeeze(), b_labels)
        total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    print(f"  Validation Loss: {avg_val_loss:.2f}")

print("Training complete!")


C:\Users\sachi\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


======== Epoch 1 / 10 ========
Training...
  Average training loss: 4.56
Running Validation...
  Validation Loss: 3.25
======== Epoch 2 / 10 ========
Training...
  Average training loss: 2.60
Running Validation...
  Validation Loss: 1.94
======== Epoch 3 / 10 ========
Training...
  Average training loss: 1.52
Running Validation...
  Validation Loss: 1.24
======== Epoch 4 / 10 ========
Training...
  Average training loss: 1.03
Running Validation...
  Validation Loss: 0.86
======== Epoch 5 / 10 ========
Training...
  Average training loss: 0.68
Running Validation...
  Validation Loss: 0.60
======== Epoch 6 / 10 ========
Training...
  Average training loss: 0.50
Running Validation...
  Validation Loss: 0.48
======== Epoch 7 / 10 ========
Training...
  Average training loss: 0.39
Running Validation...
  Validation Loss: 0.36
======== Epoch 8 / 10 ========
Training...
  Average training loss: 0.33
Running Validation...
  Validation Loss: 0.33
======== Epoch 9 / 10 ========
Training...
  Ave

In [41]:
# After training the model, save it
model_save_path = "bert_lstm_model1.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to bert_lstm_model1.pth


In [44]:
# Load the model's state dictionary
model_save_path = "bert_lstm_model1.pth"  # or the full path
if os.path.exists(model_save_path):
    model.load_state_dict(torch.load(model_save_path))
    print("Model loaded successfully!")
else:
    print(f"File not found: {model_save_path}")


C:\Users\sachi\AppData\Local\Temp\ipykernel_8440\3537068206.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))


Model loaded successfully!


In [45]:
# List of test reviews and emojis
dummy_reviews = [
    "I had a fantastic haircut! Highly recommend this salon.",  # Expected score: ~5
    "Worst service ever. Would not go back.",  # Expected score: ~1
]

dummy_emojis = ["❤️", "😡"]  # Positive and negative emojis

# Function to preprocess the review and emoji before testing
def preprocess_single_review(review, emoji, emoji_data):
    # Tokenize the review using the BERT tokenizer
    encoded = tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=64,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    
    # Convert the emoji to its corresponding index
    if emoji in emoji_data['Emoji'].values:
        emoji_index = emoji_data[emoji_data['Emoji'] == emoji]['Emoji_index'].values[0]
    else:
        emoji_index = emoji_encoder.transform(['😊'])[0]  # Default emoji if not found

    emoji_index = torch.tensor([[emoji_index]]).to(device)
    
    return input_ids, attention_mask, emoji_index

# Testing the model with dummy reviews
for review, emoji in zip(dummy_reviews, dummy_emojis):
    input_ids, attention_mask, emoji_index = preprocess_single_review(review, emoji, emoji_data)
    
    # Forward pass through the model (in evaluation mode)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        predicted_score = model(input_ids, attention_mask=attention_mask, emoji_indices=emoji_index)

    # Output the predicted score
    predicted_score = predicted_score.squeeze().item()  # Get the scalar value
    
    # Print the result
    print(f"Review: {review}")
    print(f"Emoji: {emoji}")
    print(f"Predicted Score: {predicted_score:.2f} / 5")  # The predicted score out of 5
    print()


Review: I had a fantastic haircut! Highly recommend this salon.
Emoji: ❤️
Predicted Score: 0.51 / 5

Review: Worst service ever. Would not go back.
Emoji: 😡
Predicted Score: 0.55 / 5



In [46]:
# List of negative words and emojis for deduction
negative_words = ['bad', 'worst', 'terrible', 'unprofessional', 'poor', 'horrible']
negative_emojis = ['😡', '😔', '😢', '😭', '😞', '😠']

# Test the model with dummy reviews
dummy_reviews = [
    "I had a fantastic haircut! Highly recommend this salon.",  # Expected score: ~5
    "I had a fantastic haircut! Highly recommend this salon",  # Expected score: ~1
]

dummy_emojis = ["❤️", "😡"]  # Positive and negative emojis

for review, emoji in zip(dummy_reviews, dummy_emojis):
    input_ids, attention_mask, emoji_index = preprocess_single_review(review, emoji, emoji_data)
    
    # Forward pass through the model
    with torch.no_grad():
        predicted_score = model(input_ids, attention_mask=attention_mask, emoji_indices=emoji_index)
    
    # Get the predicted score
    predicted_score = predicted_score.squeeze().item()

    # Check for negative words in the review
    for word in negative_words:
        if word in review.lower():
            predicted_score -= 1  # Deduct 1 point for negative words

    # Check for negative emoji
    if emoji in negative_emojis:
        predicted_score -= 1  # Deduct 1 point for negative emojis

    # Ensure the score stays between 0 and 5
    predicted_score = max(0, min(predicted_score, 5))
    
    # Print the result
    print(f"Review: {review}")
    print(f"Emoji: {emoji}")
    print(f"Predicted Score: {predicted_score:.2f} / 5")  # Final adjusted score
    print()


Review: I had a fantastic haircut! Highly recommend this salon.
Emoji: ❤️
Predicted Score: 0.51 / 5

Review: I had a fantastic haircut! Highly recommend this salon
Emoji: 😡
Predicted Score: 0.00 / 5



In [38]:
# List of negative words and emojis for penalty
negative_words = ['worst', 'bad', 'terrible', 'unprofessional', 'poor', 'horrible']
negative_emojis = ['😡', '😔', '😢', '😭', '😞', '😠']

# Test the model with reviews and emojis
dummy_reviews = [
    "I had a fantastic haircut! Highly recommend this salon.",  # Expected score: High
    "Worst service ever. Would not go back.",  # Expected score: Low
]

dummy_emojis = ["❤️", "😡"]  # Positive and negative emojis

for review, emoji in zip(dummy_reviews, dummy_emojis):
    input_ids, attention_mask, emoji_index = preprocess_single_review(review, emoji, emoji_data)
    
    # Forward pass through the model
    with torch.no_grad():
        predicted_score = model(input_ids, attention_mask=attention_mask, emoji_indices=emoji_index)
    
    # Get the predicted score
    predicted_score = predicted_score.squeeze().item()
    
    # Apply penalty for negative words and emojis
    for word in negative_words:
        if word in review.lower():
            predicted_score -= 0.5  # Deduct a fixed amount for negative words
    
    if emoji in negative_emojis:
        predicted_score -= 0.5  # Deduct for negative emoji
    
    # Ensure score stays between 0 and 5
    predicted_score = max(0, min(predicted_score, 5))
    
    # Print the result
    print(f"Review: {review}")
    print(f"Emoji: {emoji}")
    print(f"Predicted Score: {predicted_score:.2f} / 5")
    print()


Review: I had a fantastic haircut! Highly recommend this salon.
Emoji: ❤️
Predicted Score: 2.48 / 5

Review: Worst service ever. Would not go back.
Emoji: 😡
Predicted Score: 1.58 / 5



In [111]:
# Test the model with dummy reviews
dummy_reviews = [
    "I had a fantastic haircut! Highly recommend this salon.",  # Expected score: high
    "Worst service ever. Would not go back.",  # Expected score: low
]

dummy_emojis = ["❤️", "😡"]  # Positive and negative emojis

for review, emoji in zip(dummy_reviews, dummy_emojis):
    input_ids, attention_mask, emoji_index = preprocess_single_review(review, emoji, emoji_data)
    
    # Forward pass through the model (in evaluation mode)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        predicted_score = model(input_ids, attention_mask=attention_mask, emoji_indices=emoji_index)

    # Output the predicted score
    predicted_score = predicted_score.squeeze().item()  # Get the scalar value
    
    # Print the result
    print(f"Review: {review}")
    print(f"Emoji: {emoji}")
    print(f"Predicted Score: {predicted_score:.2f} / 5")  # The predicted score out of 5
    print()


Review: I had a fantastic haircut! Highly recommend this salon.
Emoji: ❤️
Predicted Score: 0.09 / 5

Review: Worst service ever. Would not go back.
Emoji: 😡
Predicted Score: 0.09 / 5

